**39\. Methodenseminar - Big Data Module II: Introduction to Social Network Science with Python**

# A tutorial on stochastic block modelling: SBM inference

**Author**: <a href='https://marcosoliveira.info/'>Marcos Oliveira</a>, GESIS - Leibniz Institute for the Social Sciences

**Version**: 29 May 2019

**Description**: This is an introduction to stochastic block modelling in Python using the <code>graph-tool</code> library. 
    
## Imports and Settings

In [ ]:
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np

<div class="alert alert-danger">
    You need the <code>graph_tool</code> library in your machine. Sometimes the installation is challenging. This <a href=https://git.skewed.de/count0/graph-tool/wikis/installation-instructions>page</a> might help you.
</div>

In [ ]:
import graph_tool.all as gt

In [ ]:
# a helper for colors
def node_colors(g, group_memberships):
    colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6']
    to_rgb = lambda h: list(int(h[i:i+2], 16)/255. for i in (0, 2, 4))
    vertex_color = g.new_vertex_property('vector<double>')
    for m, v in zip(group_memberships, g.vertices()):
        vertex_color[g.vertex(v)] = tuple(to_rgb(colors[m%len(colors)][1:]) + [255.0])
    return vertex_color

# blockmodel generator
def generate_assortative_block_mix(group_sizes, p_ii, p_ij):
    groups = len(group_sizes)
    membership = np.concatenate([
        np.repeat(i, g) for (i, g) in zip(range(groups), group_sizes)
    ])
    propensity = np.zeros([groups, groups])
    for i in range(groups):
        for j in range(groups):
            if i == j:
                propensity[i, j] = p_ii * group_sizes[i] * group_sizes[j] 
            else:
                propensity[i, j] = p_ij * group_sizes[i] * group_sizes[j] / 2.    
    return membership, propensity

# a helper to reset the rng
def reset_rng():
    np.random.seed(42)
    gt.seed_rng(42)
    
def block_assignment(membership):
    b = np.zeros(len(np.concatenate(list(membership.values()))), dtype=int)
    for b_ in membership:
        b[membership[b_]] = b_
    return b

# Inferring large-scale structures

# The Stochastic Block Model

Let's start with a simple graph.

In [ ]:
reset_rng()
group_sizes = [10, 10]
simple_graph = gt.generate_sbm(*generate_assortative_block_mix(group_sizes, 1.0, 0.2))
gt.remove_parallel_edges(simple_graph)
gt.remove_self_loops(simple_graph)
pos = gt.sfdp_layout(simple_graph)
gt.graph_draw(simple_graph, output_size=(400, 400), vertex_text=simple_graph.vertex_index, vertex_font_size=15, pos=pos);

In [ ]:
# membership 1
b1 = block_assignment({0: [1, 5, 2, 3, 4, 6, 7, 8],
                       1: [0, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]})
gt.graph_draw(simple_graph, output_size=(200, 200), vertex_fill_color=node_colors(simple_graph, b1), 
              vertex_text=simple_graph.vertex_index, vertex_font_size=10, pos=pos);    

In [ ]:
# membership 2
b2 = block_assignment({0: [0, 1, 2, 3, 4, 6, 7, 8, 9, 5, 10, 18],
                       1: [12, 13, 14, 15, 16, 17, 19, 11]})
gt.graph_draw(simple_graph, output_size=(200, 200), vertex_fill_color=node_colors(simple_graph, b2), 
              vertex_text=simple_graph.vertex_index, vertex_font_size=10, pos=pos);    

In [ ]:
# membership 3
b3 = block_assignment({0: [0, 1, 2, 3, 4, 6, 7, 8, 9, 5],
                       1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]})
gt.graph_draw(simple_graph, output_size=(200, 200), vertex_fill_color=node_colors(simple_graph, b3), 
              vertex_text=simple_graph.vertex_index, vertex_font_size=10, pos=pos);    

In [ ]:
print("b1 =", b1)
print("b2 =", b2)
print("b3 =", b3)

For a network $G$, we have three different $\mathbf{b}$. Which one is the best to explain our data?

<div class="alert alert-info">
Did you notice that our question is different in the case of modularity maximization?
</div>

## The entropy of stochastic blockmodel ensembles
To answer this question, first let's think a bit about our models. Specifically, let's examine the following $e_{rs}$ matrices:

In [ ]:
group_sizes = [200, 200]

In [ ]:
h_ii, h_ij = 0.1, 0.1
membership, propensity = generate_assortative_block_mix(group_sizes, h_ii, h_ij)
propensity

In [ ]:
h_ii, h_ij = 0.1, 0.01
membership, propensity = generate_assortative_block_mix(group_sizes, h_ii, h_ij)
propensity

In [ ]:
h_ii, h_ij = 0.1, 0.001
membership, propensity = generate_assortative_block_mix(group_sizes, h_ii, h_ij)
propensity

Intuitively, which of the above matrices do specify the highest number of graphs? 

Plots will help us:

In [ ]:
h_ii, h_ij = 0.1, 0.1
g = gt.generate_sbm(*generate_assortative_block_mix(group_sizes, h_ii, h_ij))
gt.graph_draw(g, output_size=(300, 300));    

h_ii, h_ij = 0.1, 0.01
g = gt.generate_sbm(*generate_assortative_block_mix(group_sizes, h_ii, h_ij))
gt.graph_draw(g, output_size=(300, 300));    

h_ii, h_ij = 0.1, 0.001
g = gt.generate_sbm(*generate_assortative_block_mix(group_sizes, h_ii, h_ij))
gt.graph_draw(g, output_size=(300, 300));    

We denote the number of graphs in a ensemble as $\Omega$. Finding out this number is a combinatorial problem that we will skip here. 

Yet, note that the third $e_{rs}$ allows less edges between groups, decreasing the possibilities of graph instances. Note as well that a graph from this matrix seems to be more 'ordered'&mdash;we can clearly see two groups. 

This order is related to the the so-called *microcanonical enropy of stochastic blockmodel ensembles*, defined as $\mathcal{S} = \ln \Omega$. Low entropy values indicate more order in the ensemble. 

### How is this related to finding out the best group assignment? 

Given a network realization and a group assignment, we can define the log-likelihood function $\mathcal{L} = \ln \mathcal{P}$, where $\mathcal{P}$ is the probability of observing the network. This function can guide us to find the most likely group assigment given a network. 

If we assume that each graph in an ensemble has the same probability, $\mathcal{P} = 1/\Omega$. Therefore, $\mathcal{L} = - \mathcal{S}$. 

That is, maximizing the log-likelihood function is equivalent to minimize the entropy. 


<div class="alert alert-info">
Can you see the diference between this approach and modularity maximization?
</div>


Let's use `graph-tool` to measure the entropy:

In [ ]:
h_ii, h_ij = 0.1, 0.001
membership, propensity = generate_assortative_block_mix(group_sizes, h_ii, h_ij)
g = gt.generate_sbm(membership, propensity)
gt.graph_draw(g, output_size=(300, 300), vertex_fill_color=node_colors(g, membership));    

The `BlockState` class represents the stochastic block model state of a graph (documentation [here](https://graph-tool.skewed.de/static/doc/inference.html#graph_tool.inference.blockmodel.BlockState)). We can set the state as the follows:

In [ ]:
block_state = gt.BlockState(g, membership, deg_corr=False)

This class has a useful method:
  - <code>gt.BlockState.<b>entropy</b>()</code>: Calculate the entropy associated with the current block partition.
  
For now, we use this with the following parameters: 
  - <code>gt.BlockState.<b>entropy</b>(adjacency=True, dl=False, dense=True, multigraph=False, exact=True)</code>

In [ ]:
block_state.entropy(adjacency=True, dl=False, dense=True, multigraph=False, exact=True)

Let's create this helper function that will make our life easier:

In [ ]:
def entropy(g, membership):
    block_state = gt.BlockState(g, membership, deg_corr=False)
    return block_state.entropy(adjacency=True, dl=False, dense=True, multigraph=False, exact=True)

Let's do the same for another network:

In [ ]:
h_ii, h_ij = 0.1, 0.1
membership, propensity = generate_assortative_block_mix(group_sizes, h_ii, h_ij)
g = gt.generate_sbm(membership, propensity)
gt.graph_draw(g, output_size=(300, 300), vertex_fill_color=node_colors(g, membership));    

In [ ]:
entropy(g, membership)

Our initial intuition made quite sense.


## Let's get back to our initial problem

We have a network and want to know which membership assignment explains best the data.

In [ ]:
# b1
gt.graph_draw(simple_graph, output_size=(200, 200), vertex_fill_color=node_colors(simple_graph, b1), pos=pos);  

# b2
gt.graph_draw(simple_graph, output_size=(200, 200), vertex_fill_color=node_colors(simple_graph, b2), pos=pos);  

# b3
gt.graph_draw(simple_graph, output_size=(200, 200), vertex_fill_color=node_colors(simple_graph, b3), pos=pos);  

Entropy:

In [ ]:
plt.figure(figsize=(3, 2))
plt.bar(range(3), [entropy(simple_graph, b1), 
                   entropy(simple_graph, b2), 
                   entropy(simple_graph, b3)], 
        width=0.7, edgecolor='k')
plt.xticks(range(3), ['b1', 'b2', 'b3']);
plt.ylabel(r'Entropy $\mathcal{S}$');
plt.xlabel('Membership');

Let's check a random assigment:

In [ ]:
membership_shuffled = b1.copy()
np.random.shuffle(membership_shuffled)
reset_rng()
gt.graph_draw(simple_graph, output_size=(200, 200),vertex_fill_color=node_colors(simple_graph, membership_shuffled));  

In [ ]:
plt.figure(figsize=(3, 2))
plt.bar(range(4), [entropy(simple_graph, b1), 
                   entropy(simple_graph, b2), 
                   entropy(simple_graph, b3), 
                   entropy(simple_graph, membership_shuffled)], width=0.7, edgecolor='k')
plt.xticks(range(4), ['b1', 'b2', 'b3', 'random']);
plt.ylabel(r'Entropy $\mathcal{S}$');
plt.xlabel('Membership');

The third membership is the most likely one from the all four. 

## Inferring modular network structure

Given that we know $B$, let's minimize the entropy.

In [ ]:
g = gt.collection.data["polbooks"]  # polbooks, football
g = gt.GraphView(g, vfilt=gt.label_largest_component(g))
g = gt.Graph(g, prune=True)
gt.remove_parallel_edges(g)
gt.remove_self_loops(g)

In [ ]:
g.num_edges(), g.num_vertices()

In [ ]:
pos = gt.sfdp_layout(g)
gt.graph_draw(g, output_size=(400, 400), pos=pos);  

How many blocks?

In [ ]:
number_of_blocks = 2

Let's use the following function from `graph-tool`:  
  - <code>gt.<b>minimize_blockmodel_dl</b>()</code>: Find the best partition.
  
For now, we will this function with the paramaters as follows:
  - <code>gt.<b>minimize_blockmodel_dl</b>(g, deg_corr=False, B_max=number_of_blocks, B_min=number_of_blocks)</code>: Find the best partition with `number_of_blocks` blocks in the graph `g`. 


In [ ]:
state = gt.minimize_blockmodel_dl(g, deg_corr=False, B_max=number_of_blocks, B_min=number_of_blocks)

Some other useful methods from BlockState:
 - <code>gt.BlockState.<b>get_blocks</b>()</code> 
 - <code>gt.BlockState.<b>get_matrix</b>()</code> 

In [ ]:
b = state.get_blocks()

In [ ]:
gt.graph_draw(g, output_size=(400, 400), pos=pos, vertex_fill_color=node_colors(g, b.a));  

Let's see the adjacency matrix and organize it based on the blocks:

In [ ]:
adjacency = gt.adjacency(g).todense()
plt.matshow(adjacency);

Ordering based on the blocks:

In [ ]:
membership_dict = {}
for i, k in zip(range(len(b.a)), b.a):
    if k not in membership_dict:
        membership_dict[k] = []
    membership_dict[k].append(i)
order = np.concatenate(list(membership_dict.values()))

In [ ]:
plt.matshow(adjacency[order][:, order])

The block matrix:

In [ ]:
plt.matshow(state.get_matrix().todense(), cmap=plt.cm.jet);

How many blocks?

## How many blocks?

Can we choose $B$ using entropy? 

In [ ]:
number_of_blocks_list = range(2, 50)
entropies = []
for n_b_i in number_of_blocks_list:
    state = gt.minimize_blockmodel_dl(g, deg_corr=False, B_max=n_b_i, B_min=n_b_i)
    entropies.append(state.entropy(adjacency=True, dl=False, dense=True, multigraph=False, exact=True))

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(number_of_blocks_list, entropies, marker="o")
plt.xlabel("Number of blocks")
plt.ylabel("Entropy");

Note that the entropy neglects the complexity of the model itself. It only measures the degree of order of an ensemble. 

Therefore, as $B$ increases, we start to memorize data (or noise), and we will have more order, so lower entropy. At the same time, we have a more complex model.

## Minimum description length
A way to account for the model complexity is to measure the description length. The description length provides us a way to measure the total amount of information required to describe data using an specific model. It is the number of bits required to send the compressed message plus the number of the encoding scheme. 



To calculate the description length, we just set `dl` to <b>`True`</b>, as follows:

In [ ]:
state.entropy(adjacency=True, dl=True, dense=True, multigraph=False, exact=True)

In [ ]:
number_of_blocks_list = range(2, 10)
entropies = []
for n_b_i in number_of_blocks_list:
    state = gt.minimize_blockmodel_dl(g, deg_corr=False, B_max=n_b_i, B_min=n_b_i)
    entropies.append(state.entropy(adjacency=True, dl=True, dense=True, multigraph=False, exact=True))

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(number_of_blocks_list, entropies, marker="o")
plt.xlabel("Number of blocks")
plt.ylabel("Entropy");

## Minimizing block model description length

With `graph-tool`, we can minimize the description length of our model. 

In [ ]:
state = gt.minimize_blockmodel_dl(g, deg_corr=False)

In [ ]:
state.B

In [ ]:
b = state.get_blocks()

gt.graph_draw(g, output_size=(400, 400), pos=pos, vertex_fill_color=node_colors(g, b.a));  

In [ ]:
membership_dict = {}
for i, k in zip(range(len(b.a)), b.a):
    if k not in membership_dict:
        membership_dict[k] = []
    membership_dict[k].append(i)
order = np.concatenate(list(membership_dict.values()))
plt.matshow(adjacency[order][:, order]);

In [ ]:
plt.matshow(state.get_matrix().todense(), cmap=plt.cm.jet);

### Degree heterogeneity

In [ ]:
g = gt.collection.data["polbooks"]
g.num_edges(), g.num_vertices()

In [ ]:
pos = gt.sfdp_layout(g)
gt.graph_draw(g, output_size=(400, 400), pos=pos);  

In [ ]:
state = gt.minimize_blockmodel_dl(g, deg_corr=False)
b = state.get_blocks()
gt.graph_draw(g, output_size=(400, 400), pos=pos, vertex_fill_color=node_colors(g, b.a));  
state.B

In [ ]:
plt.matshow(state.get_matrix().todense(), cmap=plt.cm.jet);

The well-connected nodes can influence the block assignment.

In [ ]:
degrees = [i for i in g.degree_property_map('total')]

In [ ]:
plt.figure(figsize=(5, 3))
plt.hist(degrees, bins=20, edgecolor='k')
plt.xlabel('Degree')
plt.ylabel('Frequency');

The degree-corrected stochastic block model includes the degree sequence in the model.

In [ ]:
state = gt.minimize_blockmodel_dl(g, deg_corr=True)
b = state.get_blocks()
gt.graph_draw(g, output_size=(400, 400), pos=pos, vertex_fill_color=node_colors(g, b.a));  
state.B

In [ ]:
plt.matshow(state.get_matrix().todense(), cmap=plt.cm.jet);

In [ ]:
membership_dict = {}
for i, k in zip(range(len(b.a)), b.a):
    if k not in membership_dict:
        membership_dict[k] = []
    membership_dict[k].append(i)
order = np.concatenate(list(membership_dict.values()))
plt.matshow(adjacency[order][:, order]);